## Step 1. 加载模型与Tokenizer

In [1]:
import os
import sys
from transformers import AutoTokenizer

sys.path.append(os.path.abspath('../..'))
from sfl.model.gpt2.gpt2_split import GPT2SplitLMHeadModel


cache_dir = '/root/autodl-tmp/sfl/models'  # 模型的缓存位置，需要修改
save_dir = '/root/autodl-tmp/sfl/models/checkpoints'
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=cache_dir)
model = GPT2SplitLMHeadModel.from_pretrained("gpt2", cache_dir=cache_dir)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 50256

In [7]:
# 恢复的评价指标选用ROUGE
from rouge import Rouge

def calculate_rouge(tok, logits, batch):
    my_rouge = Rouge()
    output_texts = [tok.decode(logits.argmax(dim=-1)[i], skip_special_tokens=True) for i in
                    range(len(logits))]
    hyps_and_refs = zip(output_texts, batch['input_text'])
    hyps, refs = zip(*hyps_and_refs)
    try:
        result = my_rouge.get_scores(hyps, refs, avg=True, ignore_empty=True)  # 取一个 batch 的平均
    except:
        result = {'rouge-1': {'f': 0.0, 'p': 0.0, 'r': 0.0},
                  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
                  'rouge-l': {'f': 0.0, 'p': 0.0, 'r': 0.0}}
    return result


def evaluate(epc, md, attacker, tok, test_data_loader):
    md.eval()
    attacker.eval()
    dl_len = len(test_data_loader)
    with torch.no_grad():
        rouge_1, rouge_2, rouge_l_f1, rouge_l_p, rouge_l_r = 0, 0, 0, 0, 0
        for step, batch in tqdm(enumerate(test_data_loader), total=dl_len):
            input_ids = batch['input_ids'].to(md.device)
            attention_mask = batch['input_att_mask'].to(md.device)
            inter = md(input_ids=input_ids, attention_mask=attention_mask)
            logits = attacker(inter)
            result = calculate_rouge(tokenizer, logits, batch)
            rouge_1 += result['rouge-1']['f']
            rouge_2 += result['rouge-2']['f']
            rouge_l_f1 += result['rouge-l']['f']
            rouge_l_p += result['rouge-l']['p']
            rouge_l_r += result['rouge-l']['r']
    print(
        f'Epoch {epc} Rouge_1: {rouge_1 / dl_len}, Rouge_2: {rouge_2 / dl_len}, Rouge_l_f1: {rouge_l_f1 / dl_len}, Rouge_l_p: {rouge_l_p / dl_len}, Rouge_l_r: {rouge_l_r / dl_len}')
    path = save_dir + '/attacker/'.format(epc, rouge_l_f1 / dl_len)
    os.makedirs(path, exist_ok=True)
    torch.save(attacker.state_dict(), path+'epoch_{}_rouge_{}.pt')
    md.train(True)
    attacker.train(True)
    return rouge_1 / dl_len, rouge_2 / dl_len, rouge_l_f1 / dl_len, rouge_l_p / dl_len, rouge_l_r / dl_len

### 加载数据集

In [8]:
from datasets import load_dataset
from torch.utils.data import DataLoader

def encode(examples):
    # same input and output
    input = tokenizer(examples["goal"], padding="max_length")
    return {'input_ids': input['input_ids'], 'input_att_mask': input['attention_mask'],
            'output_ids': input['input_ids'], 'output_att_mask': input['attention_mask'],
            "input_text": examples["goal"], "output_text": examples["sol1"]}

dataset = load_dataset('piqa')['train']
dataset_test = load_dataset('piqa')['validation']
dataset = dataset.map(encode)
dataset_test = dataset_test.map(encode)
dataset.set_format(type="torch", columns=["input_ids", "input_att_mask", "output_ids", "output_att_mask", "input_text"])
dataset_test.set_format(type="torch",
                        columns=["input_ids", "input_att_mask", "output_ids", "output_att_mask", "input_text"])
dataloader = DataLoader(dataset, batch_size=6)
dataloader_test = DataLoader(dataset_test, batch_size=6)


### 切分模型

In [9]:
from sfl.utils import FLConfig
model.config_sfl(FLConfig(collect_intermediates=False,
                          split_point_1=2, # 第0～1层为top，余下的都是trunk
                          split_point_2=999,
                          attack_mode='b2tr' # 攻击的输出是bottom-to-trunk中间输出
                          ),
                 param_keeper=None)
# model = model.convert_to_lora_model(restore_top_bottom=False)
model.print_split_model()

=================Split-gpt2=================
==================Top Layers==================
==================Trunk Layers==================

transformer.h.2:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bias: (768,), mlp.c_fc.weight: (768, 3072), mlp.c_fc.bias: (3072,), mlp.c_proj.weight: (3072, 768), mlp.c_proj.bias: (768,)]

transformer.h.3:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bias: (768,), mlp.c_fc.weight: (768, 3072), mlp.c_fc.bias: (3072,), mlp.c_proj.weight: (3072, 768), mlp.c_proj.bias: (768,)]

transformer.h.4:[ln_1.weight: (768,), ln_1.bias: (768,), attn.c_attn.weight: (768, 2304), attn.c_attn.bias: (2304,), attn.c_proj.weight: (768, 768), attn.c_proj.bias: (768,), ln_2.weight: (768,), ln_2.bia

### 训练Attack Model

In [10]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from sfl.model.attack_model import GPT2AttackModel
from sfl.utils import get_best_gpu
from tqdm.notebook import tqdm
import torch


def get_output(text, encoder_model, attack_model):
    t = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    inter = encoder_model(t['input_ids'].to(device), attention_mask=t['attention_mask'].to(device))
    res = attack_model(inter)
    r = tokenizer.decode(res.argmax(dim=-1)[-1], skip_special_tokens=True)
    return r


def calc_loss(lm_logits, labels):
    labels = labels.to(lm_logits.device)
    # Shift so that tokens < n predict n
    shift_logits = lm_logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    # Flatten the tokens
    loss_fct = CrossEntropyLoss()
    return loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))


# 开始训练Attack Model
device = get_best_gpu()
attack_model = GPT2AttackModel(model.config)
optimizer = Adam(attack_model.parameters(), lr=1e-3)
model.to(device)
attack_model.to(device)
epoch = 20
evaluate(0,model,attack_model,tokenizer,dataloader_test)
with tqdm(total=epoch * len(dataloader)) as pbar:
    for epc in range(epoch):
        model.train(True)
        rouge_1, rouge_2, rouge_l_f1, rouge_l_p, rouge_l_r = 0, 0, 0, 0, 0
        for step, batch in enumerate(dataloader):
            optimizer.zero_grad()
            input_ids, labels = batch['input_ids'].to(device), batch['output_ids'].to(device)
            attention_mask = batch['input_att_mask'].to(device)
            intermediate = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)
            logits = attack_model(intermediate)
            loss = calc_loss(logits, labels)
            loss.backward()
            optimizer.step()
            res = calculate_rouge(tokenizer, logits, batch)
            rouge_1 += res['rouge-1']['f']
            rouge_2 += res['rouge-2']['f']
            rouge_l_f1 += res['rouge-l']['f']
            rouge_l_p += res['rouge-l']['p']
            rouge_l_r += res['rouge-l']['r']
            pbar.set_description(f'Epoch {epc} Loss {loss.item():.5f}, Rouge_1 {rouge_1 / (step + 1):.4f}')
            if (epc * len(dataloader) + step) % 200 == 0:
                q = "To mix food coloring with sugar, you can"
                print(q, "=>", get_output(q, model, attack_model))
            pbar.update(1)
        rouge_1 /= len(dataloader)
        rouge_2 /= len(dataloader)
        rouge_l_f1 /= len(dataloader)
        rouge_l_p /= len(dataloader)
        rouge_l_r /= len(dataloader)
        print(
            f'Epoch {epc} Training Rouge_1: {rouge_1}, Rouge_2: {rouge_2}, Rouge_l_f1: {rouge_l_f1}, Rouge_l_p: {rouge_l_p}, Rouge_l_r: {rouge_l_r}')
        evaluate(epc, model, attack_model, tokenizer, dataloader_test)

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 0 Rouge_1: 2.857305921240627e-05, Rouge_2: 0.0, Rouge_l_f1: 2.857305921240627e-05, Rouge_l_p: 1.9388863037071504e-05, Rouge_l_r: 5.428881650380022e-05


  0%|          | 0/53720 [00:00<?, ?it/s]

To mix food coloring with sugar, you can =>  confined podcasts alerted thwarted..." grab Tinder Tinder Tinder
To mix food coloring with sugar, you can => 
To mix food coloring with sugar, you can => 
To mix food coloring with sugar, you can =>  do? a?
To mix food coloring with sugar, you can =>  a?
To mix food coloring with sugar, you can =>  do a a a? can
To mix food coloring with sugar, you can =>  a a? can
To mix food coloring with sugar, you can =>  a of a? can
To mix food coloring with sugar, you can =>  a to of a. can
To mix food coloring with sugar, you can =>  make a to a, you can
To mix food coloring with sugar, you can =>  of to a. can
To mix food coloring with sugar, you can =>  make a.. a? can
To mix food coloring with sugar, you can =>  a. a you can
To mix food coloring with sugar, you can =>  make a.. a. can
Epoch 0 Training Rouge_1: 0.144919558869667, Rouge_2: 0.026456643927733177, Rouge_l_f1: 0.14197119266294508, Rouge_l_p: 0.22646803487458755, Rouge_l_r: 0.109945955403

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 0 Rouge_1: 0.18902886941956087, Rouge_2: 0.030795116903582567, Rouge_l_f1: 0.18629481653551247, Rouge_l_p: 0.2631737845337194, Rouge_l_r: 0.14899379668583282
To mix food coloring with sugar, you can => .. a. can
To mix food coloring with sugar, you can =>  make a. a. can
To mix food coloring with sugar, you can =>  the.. a. you can
To mix food coloring with sugar, you can =>  a to to a. can
To mix food coloring with sugar, you can =>  make a.. a. can
To mix food coloring with sugar, you can =>  make a from. a, can
To mix food coloring with sugar, you can =>  make of.. a. can
To mix food coloring with sugar, you can =>  make a to, a, can
To mix food coloring with sugar, you can =>  a in, a, can
To mix food coloring with sugar, you can =>  make a for a to can
To mix food coloring with sugar, you can =>  make a to. a, can
To mix food coloring with sugar, you can =>  make a from to a, can
To mix food coloring with sugar, you can =>  make a to to a, can
Epoch 1 Training Rouge_1: 0.188

  0%|          | 0/307 [00:00<?, ?it/s]

Epoch 1 Rouge_1: 0.20824593799649735, Rouge_2: 0.03705281939815808, Rouge_l_f1: 0.20605416718404335, Rouge_l_p: 0.27616871019558326, Rouge_l_r: 0.16818241996319805
To mix food coloring with sugar, you can =>  make butter, to a can you can
To mix food coloring with sugar, you can =>  make the, to a can can
To mix food coloring with sugar, you can =>  make a., a. can
To mix food coloring with sugar, you can =>  make a from to a, can


KeyboardInterrupt: 